<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 2: Preparing Data 

## Identifying Outliers

### Lesson Objectives

After completing this lesson, you should be able to:

- Compute the inverse of covariance matrix given of a dataset
-	Compute Mahalanobis Distance for all elements in a dataset
-	Remove outliers from a dataset


## Mahalanobis Distance 

-	Multi-dimensional generalization of measuring how many standard deviations a point is away from the mean
-	Measured along each Principal Component axis 
-	Unitless and scale-invariant 
-	Takes into account the correlations of the dataset
-	Used to detect outliers

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

import org.apache.spark.sql.functions._
import  org.apache.spark.mllib.linalg.{Vector, Vectors}
import  org.apache.spark.ml.feature.StandardScaler
import  org.apache.spark.ml.feature.VectorAssembler 
import org.apache.spark.mllib.util.MLUtils


Intitializing Scala interpreter ...

Spark Web UI available at http://host.docker.internal:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1669602291574)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3c371458
import spark.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.mllib.util.MLUtils


In [2]:
// Continuing from Previous Example 

val  dfRandom = spark.range(0, 10).select("id").
 withColumn("uniform", rand(10L)).
 withColumn("normal1", randn(10L)).
 withColumn("normal2", randn(11L))

val  assembler = new VectorAssembler().
 setInputCols(Array("uniform","normal1","normal2")).
 setOutputCol("features")

val dfVec = MLUtils.convertVectorColumnsFromML(assembler.transform(dfRandom))

22/11/28 10:25:04 WARN MLUtils: Vector column conversion has serialization overhead. Please migrate your datasets and workflows to use the spark.ml package.


dfRandom: org.apache.spark.sql.DataFrame = [id: bigint, uniform: double ... 2 more fields]
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_f3412ca989c8, handleInvalid=error, numInputCols=3
dfVec: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, uniform: double ... 3 more fields]


In [3]:
// Continuing from the Previous Example
dfVec.select("id","features").show()

// An Example with Outliers 
val dfOutlier = dfVec.select("id","features").unionAll(spark.createDataFrame(Seq((10,Vectors.dense(3, 3, 3)))))
dfOutlier.sort(dfOutlier("id").desc).show(5)

22/11/28 10:25:07 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped
+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.17094971379555...|
|  1|[0.03422639313807...|
|  2|[0.36546259581613...|
|  3|[0.41750190407920...|
|  4|[0.98991293998274...|
|  5|[0.16452185994603...|
|  6|[0.18141810315190...|
|  7|[0.49595620559530...|
|  8|[0.96974749453753...|
|  9|[0.07530606222259...|
+---+--------------------+

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|       [3.0,3.0,3.0]|
|  9|[0.07530606222259...|
|  8|[0.96974749453753...|
|  7|[0.49595620559530...|
|  6|[0.18141810315190...|
+---+--------------------+
only showing top 5 rows



dfOutlier: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, features: vector]


In [4]:
// An Example with Outliers 

val scaler = new StandardScaler().
 setInputCol("features").setOutputCol("scaledFeat").
 setWithStd(true).setWithMean(true)

val scalerModel = scaler.fit(MLUtils.convertVectorColumnsToML(dfOutlier.select("id","features")))

val dfScaled = scalerModel.transform(MLUtils.convertVectorColumnsToML(dfOutlier)).select("id","scaledFeat")
dfScaled.sort(dfScaled("id").desc).show(3)

22/11/28 10:25:10 WARN MLUtils: Vector column conversion has serialization overhead. Please migrate your datasets and workflows to use the spark.ml package.
22/11/28 10:25:11 WARN MLUtils: Vector column conversion has serialization overhead. Please migrate your datasets and workflows to use the spark.ml package.
+---+--------------------+
| id|          scaledFeat|
+---+--------------------+
| 10|[2.78349197952832...|
|  9|[-0.6429283194699...|
|  8|[0.40495304697417...|
+---+--------------------+
only showing top 3 rows



scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_87e710e8b2a1
scalerModel: org.apache.spark.ml.feature.StandardScalerModel = StandardScalerModel: uid=stdScal_87e710e8b2a1, numFeatures=3, withMean=true, withStd=true
dfScaled: org.apache.spark.sql.DataFrame = [id: bigint, scaledFeat: vector]


In [5]:
import  org.apache.spark.mllib.stat.Statistics

import  breeze.linalg._

val  rddVec = MLUtils.convertVectorColumnsFromML(dfScaled.select("scaledFeat")).rdd.map(_(0).asInstanceOf[org.apache.spark.mllib.linalg.Vector])

val  colCov = Statistics.corr(rddVec)
val  invColCovB = inv(new DenseMatrix(3, 3, colCov.toArray))

22/11/28 10:25:13 WARN MLUtils: Vector column conversion has serialization overhead. Please migrate your datasets and workflows to use the spark.ml package.
22/11/28 10:25:15 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/28 10:25:15 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


import org.apache.spark.mllib.stat.Statistics
import breeze.linalg._
rddVec: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[41] at map at <console>:40
colCov: org.apache.spark.mllib.linalg.Matrix =
1.0                 0.8817973228811657  0.7242484448333565
0.8817973228811657  1.0                 0.727215076550755
0.7242484448333565  0.727215076550755   1.0
invColCovB: breeze.linalg.DenseMatrix[Double] =
4.8119759706785326   -3.626796245237513   -0.8476052042250196
-3.626796245237513   4.85595305448813     -0.9046207319057327
-0.8476052042250195  -0.9046207319057327  2.2717305857948573


In [6]:
// Computing Mahalanobis Distance 

val mahalanobis = udf[Double, org.apache.spark.ml.linalg.Vector]{ v =>
 val k = v.toArray
 val vB = new DenseVector(k);
 vB.t * invColCovB * vB
}

val dfMahalanobis = dfScaled.withColumn("mahalanobis", mahalanobis(dfScaled("scaledFeat")))

mahalanobis: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5298/0x0000000101bb4840@32585ad1,DoubleType,List(Some(class[value[0]: vector])),Some(class[value[0]: double]),None,false,true)
dfMahalanobis: org.apache.spark.sql.DataFrame = [id: bigint, scaledFeat: vector ... 1 more field]


In [7]:
// Computing Mahalanobis Distance 
dfMahalanobis.show()

22/11/28 10:25:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/28 10:25:16 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
+---+--------------------+-------------------+
| id|          scaledFeat|        mahalanobis|
+---+--------------------+-------------------+
|  0|[-0.5308771591803...| 2.9889593550354507|
|  1|[-0.6910551371889...|  4.026861958539909|
|  2|[-0.3029959170567...| 0.5444288973785845|
|  3|[-0.2420293523590...| 2.1422662305099758|
|  4|[0.42857784091221...|0.21636673463383588|
|  5|[-0.5384077006241...| 0.9508135605939119|
|  6|[-0.5186129351105...| 1.3912647970003544|
|  7|[-0.1501163464249...| 0.7706788231143263|
|  8|[0.40495304697417...|  2.784409266013935|
|  9|[-0.6429283194699...|  6.147204454666931|
| 10|[2.78349197952832...|   8.03674592251277|
+---+--------------------+-------------------+



In [8]:
// Removing Outliers 

dfMahalanobis.sort(dfMahalanobis("mahalanobis").desc).show(2)

val ids = dfMahalanobis.select("id","mahalanobis").sort(dfMahalanobis("mahalanobis").desc).drop("mahalanobis").collect() 

val idOutliers = ids.map(_(0).asInstanceOf[Long]).slice(0,2)

+---+--------------------+-----------------+
| id|          scaledFeat|      mahalanobis|
+---+--------------------+-----------------+
| 10|[2.78349197952832...| 8.03674592251277|
|  9|[-0.6429283194699...|6.147204454666931|
+---+--------------------+-----------------+
only showing top 2 rows



ids: Array[org.apache.spark.sql.Row] = Array([10], [9], [1], [0], [8], [3], [6], [5], [7], [2], [4])
idOutliers: Array[Long] = Array(10, 9)


## Removing Outliers

In [9]:
dfOutlier.filter("id not in (10, 2)").show()

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.17094971379555...|
|  1|[0.03422639313807...|
|  3|[0.41750190407920...|
|  4|[0.98991293998274...|
|  5|[0.16452185994603...|
|  6|[0.18141810315190...|
|  7|[0.49595620559530...|
|  8|[0.96974749453753...|
|  9|[0.07530606222259...|
+---+--------------------+



## Lesson Summary

- Having completed this lesson, you should be able to:
-	Compute the inverse of covariance matrix given of a dataset 
-	Compute Mahalanobis Distance for all elements in a dataset
-	Remove outliers from a dataset

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.